In [1]:
import tensorflow as tf 
import tensorflow.keras.layers as layers
import tensorflow.keras.models as models
import tensorflow.keras.optimizers as optim

import pandas as pd 
import numpy as np 
import os

In [2]:
PATH = r"C:\Users\suyash\Desktop\toxic\tox21_dense_train.csv"
data = pd.read_csv(PATH).values
labels = pd.read_csv(PATH).columns
labels

Index(['Unnamed: 0', 'AW', 'AWeight', 'Arto', 'BertzCT', 'Chi0', 'Chi1',
       'Chi10', 'Chi2', 'Chi3',
       ...
       'W3D', 'W3DH', 'WNSA1', 'WNSA2', 'WNSA3', 'WPSA1', 'WPSA2', 'WPSA3',
       'grav', 'rygr'],
      dtype='object', length=802)

In [3]:
for x in range(799):
    if max(data[:, 3:][:, x])!=min(data[:, 3:][:, x]):
        xtr = (data[:, 3:][:, x]-np.mean(data[:, 3:][:, x]))/(max(data[:, 3:][:, x])-min(data[:, 3:][:, x]))
        data[:, 3:][:, x] = xtr
    else:
        xtr = (data[:, 3:][:, x])/(max(data[:, 3:][:, x])+min(data[:, 3:][:, x])+1e-10)+0.5
        data[:, 3:][:, x] = xtr

In [4]:
data = data[:, 3:]

In [5]:
xtrain = np.array(data[:10000], dtype="float32")
xvalid = np.array(data[10000:12000], dtype="float32")

In [6]:
def DenseBlock(x, ndim):
    x = layers.Dense(ndim)(x)
    x = layers.BatchNormalization(momentum=0.5)(x)
    x = layers.LeakyReLU(0.2)(x)
    return  x

def autoencoder(inp):
    inp = layers.Input(inp)
    x = DenseBlock(inp, 300)
    x = DenseBlock(x, 100)
    x = DenseBlock(x, 30)
    x = DenseBlock(x, 10)
    x = DenseBlock(x, 3)
    enc_out = DenseBlock(x, 1)
    x = DenseBlock(enc_out, 3)
    x = DenseBlock(x, 10)
    x = DenseBlock(x, 30)
    x = DenseBlock(x, 100)
    x = DenseBlock(x, 300)
    dec_out = DenseBlock(x, 799)
    encoder = models.Model(inputs=inp, outputs=enc_out, name="Encoder")
    model = models.Model(inputs=inp, outputs=dec_out, name="Autoencoder")

    return encoder, model

encoder, model = autoencoder((799))
print(encoder.summary())
print(model.summary())

Model: "Encoder"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 799)]             0         
                                                                 
 dense (Dense)               (None, 300)               240000    
                                                                 
 batch_normalization (BatchN  (None, 300)              1200      
 ormalization)                                                   
                                                                 
 leaky_re_lu (LeakyReLU)     (None, 300)               0         
                                                                 
 dense_1 (Dense)             (None, 100)               30100     
                                                                 
 batch_normalization_1 (Batc  (None, 100)              400       
 hNormalization)                                           

In [16]:
model.compile(loss='mse', optimizer=optim.Adam(learning_rate=1e-5), metrics="accuracy")
model.fit(xtrain, xtrain, batch_size=100, verbose=1, epochs=100, validation_data=[xvalid, xvalid], validation_batch_size=100)

Epoch 1/100
100/100 [==============================] - 4s 20ms/step - loss: 0.0027 - accuracy: 0.7645 - val_loss: 0.0029 - val_accuracy: 0.7490
Epoch 2/100
100/100 [==============================] - 1s 10ms/step - loss: 0.0027 - accuracy: 0.7613 - val_loss: 0.0029 - val_accuracy: 0.7330
Epoch 3/100
100/100 [==============================] - 1s 10ms/step - loss: 0.0027 - accuracy: 0.7619 - val_loss: 0.0028 - val_accuracy: 0.7735
Epoch 4/100
100/100 [==============================] - 1s 11ms/step - loss: 0.0027 - accuracy: 0.7673 - val_loss: 0.0028 - val_accuracy: 0.7495
Epoch 5/100
100/100 [==============================] - 1s 12ms/step - loss: 0.0027 - accuracy: 0.7688 - val_loss: 0.0028 - val_accuracy: 0.7745
Epoch 6/100
100/100 [==============================] - 1s 11ms/step - loss: 0.0027 - accuracy: 0.7683 - val_loss: 0.0028 - val_accuracy: 0.7525
Epoch 7/100
100/100 [==============================] - 1s 12ms/step - loss: 0.0027 - accuracy: 0.7738 - val_loss: 0.0028 - val_accuracy:

KeyboardInterrupt: 

In [8]:
# model.save("./model50.h5")
# encoder.save("./encoder50.h5")

In [9]:
# xtrain[1].shape

In [10]:
# out  = encoder(xtrain[1].reshape(-1,799))

In [11]:
# from minisom import MiniSom

In [12]:
# r = 100
# c = 100
# iter = 50000
# sigma = 1
# lr = 0.1

In [13]:
# som = MiniSom(x=r, y=c, input_len=50, sigma=sigma, learning_rate=lr)
# xsom = encoder(xtrain)
# som.random_weights_init(xsom)

In [14]:
# som.train(xsom, iter, verbose=1)